In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
ds = load_dataset("jahjinx/IMDb_movie_reviews")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.24k [00:00<?, ?B/s]

IMDB_train.csv:   0%|          | 0.00/47.6M [00:00<?, ?B/s]

IMDB_validation.csv:   0%|          | 0.00/5.23M [00:00<?, ?B/s]

IMDB_test.csv:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/36000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 36000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
})

In [ ]:
import pandas as pd
train_df = pd.DataFrame(ds['train'])
val_df = pd.DataFrame(ds['validation'])
test_df = pd.DataFrame(ds['test'])

In [ ]:
# remove duplicates
train_df.drop_duplicates(inplace=True)
val_df.drop_duplicates(inplace=True)
test_df.drop_duplicates(inplace=True)

In [ ]:
print(f"Shape of training data: {train_df.shape}")
print(f"Shape of validation data: {val_df.shape}")
print(f"Shape of testing data: {test_df.shape}")

Shape of training data: (35767, 2)
Shape of validation data: (3999, 2)
Shape of testing data: (9981, 2)


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure necessary NLTK data is downloaded
nltk.download('stopwords')
nltk.download('wordnet')

# Define the preprocessing function
def preprocess_comment(comment):
    # Convert to lowercase
    comment = comment.lower()

    # Remove trailing and leading whitespaces
    comment = comment.strip()

    # Remove URLs
    comment = re.sub(r'https?://\S+|www\.\S+', '', comment)

    # Remove HTML tags
    comment = re.sub(r'<.*?>', '', comment)

    # Remove newline characters
    comment = re.sub(r'\n', ' ', comment)

    # Remove non-alphanumeric characters and punctuation
    comment = re.sub(r'[^\w\s]', '', comment)


    # Remove stopwords but retain important ones for sentiment analysis
    stop_words = set(stopwords.words('english')) - {'not', 'but', 'however', 'no', 'yet'}
    comment = ' '.join([word for word in comment.split() if word not in stop_words])

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

    return comment

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
train_df['text'] = train_df['text'].apply(preprocess_comment)
val_df['text'] = val_df['text'].apply(preprocess_comment)
test_df['text'] = test_df['text'].apply(preprocess_comment)

In [ ]:
train_df.head()

,text,label
0,beautifully photographed ably acted generally ...,0
1,well start describing celluloid debacle alread...,0
2,first caught movie first run hbo probably 1981...,1
3,love umberto lenzis cop movie rome armed teeth...,0
4,generally wont review movie havent seen awhile...,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression

In [ ]:
tfidf = TfidfVectorizer(max_features=2000)

In [ ]:
X_train = tfidf.fit_transform(train_df['text'])

In [ ]:
y = train_df['label']

base_model = LogisticRegression()
base_model.fit(X_train, y)

LogisticRegression()

In [ ]:
X_val = tfidf.transform(val_df['text'])

X_test = tfidf.transform(test_df['text'])

y_val = val_df['label']
y_test = test_df['label']

In [ ]:
y_val_pred = base_model.predict(X_val)
y_test_pred = base_model.predict(X_test)
y_train_pred = base_model.predict(X_train)

In [ ]:
print(f"val_acc_tfidf: {accuracy_score(y_val, y_val_pred)}")
print(f"test_acc_tfidf: {accuracy_score(y_test, y_test_pred)}")
print(f"train_acc_tfidf: {accuracy_score(y, y_train_pred)}")

val_acc_tfidf: 0.8764691172793199
test_acc_tfidf: 0.876765855124737
train_acc_tfidf: 0.8932255990158526


In [ ]:
print(f"val: {classification_report(y_val, y_val_pred)}")
print(f"test: {classification_report(y_test, y_test_pred)}")
print(f"train: {classification_report(y, y_train_pred)}")

val:               precision    recall  f1-score   support

           0       0.88      0.87      0.88      2011
           1       0.87      0.88      0.88      1988

    accuracy                           0.88      3999
   macro avg       0.88      0.88      0.88      3999
weighted avg       0.88      0.88      0.88      3999

test:               precision    recall  f1-score   support

           0       0.89      0.86      0.88      5028
           1       0.87      0.89      0.88      4953

    accuracy                           0.88      9981
   macro avg       0.88      0.88      0.88      9981
weighted avg       0.88      0.88      0.88      9981

train:               precision    recall  f1-score   support

           0       0.90      0.88      0.89     17785
           1       0.89      0.90      0.89     17982

    accuracy                           0.89     35767
   macro avg       0.89      0.89      0.89     35767
weighted avg       0.89      0.89      0.89     35767



In [ ]:
print(f"val: {confusion_matrix(y_val, y_val_pred)}\n")
print(f"test: {confusion_matrix(y_test, y_test_pred)}\n")
print(f"train: {confusion_matrix(y, y_train_pred)}\n")

val: [[1756  255]
 [ 239 1749]]

test: [[4347  681]
 [ 549 4404]]

train: [[15704  2081]
 [ 1738 16244]]

